In [ ]:
import yaml
from pathlib import Path
import shutil

# First dataset
dataset_path = '/kaggle/input/bsed-datasets-v1/kaggle/working/Data Labeling/yolo_classification_dataset'

print("="*60)
print("📊 ANALYZING: YOLO CLASSIFICATION DATASET")
print("="*60)

# Create a working copy in writable directory
working_dir = Path('/kaggle/working')
dataset_copy_dir = working_dir / 'yolo_classification_dataset'

# Copy the dataset to working directory if not already copied
if not dataset_copy_dir.exists():
    print(f"📁 Copying dataset to {dataset_copy_dir}...")
    shutil.copytree(dataset_path, dataset_copy_dir)
    print("✅ Dataset copied to working directory")
else:
    print("✅ Using existing copy in working directory")

# Now use the copy
data_yaml_path = dataset_copy_dir / 'data.yaml'

# Load data.yaml
with open(data_yaml_path, 'r') as file:
    data_config = yaml.safe_load(file)

print(f"\nNumber of classes: {data_config['nc']}")
print(f"Class names: {data_config['names']}")

# Update paths in the working copy
if 'train' in data_config:
    # Fix path relative to current location
    train_relative = data_config['train'].replace('../', '')
    train_path = dataset_copy_dir / train_relative
    data_config['train'] = str(train_path)
    
if 'val' in data_config:
    val_relative = data_config['val'].replace('../', '')
    val_path = dataset_copy_dir / val_relative
    data_config['val'] = str(val_path)
    
if 'test' in data_config:
    test_relative = data_config['test'].replace('../', '')
    test_path = dataset_copy_dir / test_relative
    data_config['test'] = str(test_path)

# Count images
train_img_path = Path(data_config['train'])
val_img_path = Path(data_config['val'])

train_images = len(list(train_img_path.glob('*.jpg'))) + len(list(train_img_path.glob('*.png')))
val_images = len(list(val_img_path.glob('*.jpg'))) + len(list(val_img_path.glob('*.png')))

if 'test' in data_config:
    test_img_path = Path(data_config['test'])
    test_images = len(list(test_img_path.glob('*.jpg'))) + len(list(test_img_path.glob('*.png')))
else:
    test_images = 0

print(f"\n📈 Dataset Statistics:")
print(f"Training images: {train_images}")
print(f"Validation images: {val_images}")
print(f"Test images: {test_images}")

# Update and save yaml - Now it's writable!
data_config['path'] = str(dataset_copy_dir)
with open(data_yaml_path, 'w') as f:
    yaml.dump(data_config, f, default_flow_style=False)

print(f"\n✅ Dataset configuration saved to: {data_yaml_path}")
print(f"Working directory: {dataset_copy_dir}")

In [ ]:
import yaml
from pathlib import Path
import shutil

# First dataset
dataset_path = '/kaggle/input/roboflow-datasets/kaggle/working/Black-Sigatoka-4'

print("="*60)
print("📊 ANALYZING: YOLO CLASSIFICATION DATASET")
print("="*60)

# Create a working copy in writable directory
working_dir = Path('/kaggle/working')
dataset_copy_dir = working_dir / 'Black-Sigatoka-4'

# Copy the dataset to working directory if not already copied
if not dataset_copy_dir.exists():
    print(f"📁 Copying dataset to {dataset_copy_dir}...")
    shutil.copytree(dataset_path, dataset_copy_dir)
    print("✅ Dataset copied to working directory")
else:
    print("✅ Using existing copy in working directory")

# Now use the copy
data_yaml_path = dataset_copy_dir / 'data.yaml'

# Load data.yaml
with open(data_yaml_path, 'r') as file:
    data_config = yaml.safe_load(file)

print(f"\nNumber of classes: {data_config['nc']}")
print(f"Class names: {data_config['names']}")

# Update paths in the working copy
if 'train' in data_config:
    # Fix path relative to current location
    train_relative = data_config['train'].replace('../', '')
    train_path = dataset_copy_dir / train_relative
    data_config['train'] = str(train_path)
    
if 'val' in data_config:
    val_relative = data_config['val'].replace('../', '')
    val_path = dataset_copy_dir / val_relative
    data_config['val'] = str(val_path)
    
if 'test' in data_config:
    test_relative = data_config['test'].replace('../', '')
    test_path = dataset_copy_dir / test_relative
    data_config['test'] = str(test_path)

# Count images
train_img_path = Path(data_config['train'])
val_img_path = Path(data_config['val'])

train_images = len(list(train_img_path.glob('*.jpg'))) + len(list(train_img_path.glob('*.png')))
val_images = len(list(val_img_path.glob('*.jpg'))) + len(list(val_img_path.glob('*.png')))

if 'test' in data_config:
    test_img_path = Path(data_config['test'])
    test_images = len(list(test_img_path.glob('*.jpg'))) + len(list(test_img_path.glob('*.png')))
else:
    test_images = 0

print(f"\n📈 Dataset Statistics:")
print(f"Training images: {train_images}")
print(f"Validation images: {val_images}")
print(f"Test images: {test_images}")

# Update and save yaml - Now it's writable!
data_config['path'] = str(dataset_copy_dir)
with open(data_yaml_path, 'w') as f:
    yaml.dump(data_config, f, default_flow_style=False)

print(f"\n✅ Dataset configuration saved to: {data_yaml_path}")
print(f"Working directory: {dataset_copy_dir}")

# **COMBINE DATASETS WITH CLASS MAPPING**

In [ ]:
import yaml
from pathlib import Path
import shutil
from tqdm import tqdm
import os
import re

print("="*80)
print("🔄 COMBINING BSED AND ROBoFLOW DATASETS")
print("="*80)

# ============================================================================
# CLASS MAPPING CONFIGURATION
# ============================================================================
# Final classification scheme (from data-labeling-classification.ipynb)
FINAL_CLASS_ORDER = ['Healthy', 'Stage1', 'Stage2', 'Stage3', 'Stage4', 'Stage5', 'Stage6']

# Roboflow class mapping: actual class names → final class names
# Based on actual Roboflow dataset class names
ROBOFLOW_CLASS_NAME_MAPPING = {
    'Functional': 'Healthy',   # Functional → Healthy
    'Mild': 'Stage4',           # Mild → Stage4
    'Moderate': 'Stage5',       # Moderate → Stage5
    'Severe': 'Stage6'          # Severe → Stage6
}

# Also map numeric IDs (used in label files) → class names
# Label files use numeric IDs (0, 1, 2, 3) which correspond to:
ROBOFLOW_CLASS_ID_MAPPING = {
    '0': 'Functional',   # Label file ID 0 → Functional → Healthy
    '1': 'Mild',          # Label file ID 1 → Mild → Stage4
    '2': 'Moderate',       # Label file ID 2 → Moderate → Stage5
    '3': 'Severe'          # Label file ID 3 → Severe → Stage6
}

# Create mapping: class_name → final_class_id
FINAL_CLASS_TO_ID = {cls: idx for idx, cls in enumerate(FINAL_CLASS_ORDER)}

print("\n📋 Class Mapping Configuration:")
print(f"   Final class order: {FINAL_CLASS_ORDER}")
print(f"\n   Roboflow class name mapping:")
for old_name, new_name in ROBOFLOW_CLASS_NAME_MAPPING.items():
    new_id = FINAL_CLASS_TO_ID[new_name]
    print(f"      {old_name} → {new_name} (Final ID: {new_id})")
print(f"\n   Roboflow label file ID mapping:")
for old_id, class_name in ROBOFLOW_CLASS_ID_MAPPING.items():
    final_name = ROBOFLOW_CLASS_NAME_MAPPING[class_name]
    final_id = FINAL_CLASS_TO_ID[final_name]
    print(f"      ID {old_id} ({class_name}) → {final_name} (Final ID: {final_id})")

# ============================================================================
# DATASET PATHS
# ============================================================================
working_dir = Path('/kaggle/working')

# BSED dataset (already copied)
bsed_dataset_dir = working_dir / 'yolo_classification_dataset'

# Roboflow dataset (already copied)
roboflow_dataset_dir = working_dir / 'Black-Sigatoka-4'

# Combined dataset output
combined_dataset_dir = working_dir / 'combined_yolo_dataset'

print(f"\n📁 Dataset Paths:")
print(f"   BSED: {bsed_dataset_dir}")
print(f"   Roboflow: {roboflow_dataset_dir}")
print(f"   Combined Output: {combined_dataset_dir}")

# Validate datasets exist
if not bsed_dataset_dir.exists():
    raise FileNotFoundError(f"BSED dataset not found: {bsed_dataset_dir}")
if not roboflow_dataset_dir.exists():
    raise FileNotFoundError(f"Roboflow dataset not found: {roboflow_dataset_dir}")

# Create combined dataset structure
splits = ['train', 'valid', 'test']
for split in splits:
    (combined_dataset_dir / split / 'images').mkdir(parents=True, exist_ok=True)
    (combined_dataset_dir / split / 'labels').mkdir(parents=True, exist_ok=True)

print(f"\n✅ Combined dataset directory structure created")


In [ ]:
# ============================================================================
# FUNCTION: Update label file with new class mapping
# ============================================================================
def update_label_file(label_path, old_to_new_class_mapping):
    """
    Update YOLO label file with new class IDs.
    
    Args:
        label_path: Path to .txt label file
        old_to_new_class_mapping: Dict mapping old_class_id → new_class_id
    
    Returns:
        True if updated, False if no changes needed
    """
    try:
        with open(label_path, 'r') as f:
            lines = f.readlines()
        
        updated_lines = []
        changed = False
        
        for line in lines:
            parts = line.strip().split()
            if len(parts) >= 5:  # YOLO format: class_id x y w h
                old_class_id = parts[0]
                
                # Check if this class needs mapping
                if old_class_id in old_to_new_class_mapping:
                    new_class_id = old_to_new_class_mapping[old_class_id]
                    parts[0] = str(new_class_id)
                    changed = True
                
                updated_lines.append(' '.join(parts) + '\n')
            else:
                updated_lines.append(line)
        
        if changed:
            with open(label_path, 'w') as f:
                f.writelines(updated_lines)
        
        return changed
    except Exception as e:
        print(f"Error updating {label_path}: {e}")
        return False

# ============================================================================
# FUNCTION: Rename image file with new class name
# ============================================================================
def rename_image_with_class(filename, class_name_mapping):
    """
    Rename image filename by replacing old class names with new ones.
    
    Args:
        filename: Original filename (e.g., "Functional-1-_png.rf.xxx.jpg")
        class_name_mapping: Dict mapping old_class_name → new_class_name
    
    Returns:
        New filename with updated class name
    """
    new_filename = filename
    
    # Replace class names in filename (case-insensitive)
    for old_name, new_name in class_name_mapping.items():
        # Replace both capitalized and lowercase versions
        if old_name in new_filename:
            new_filename = new_filename.replace(old_name, new_name)
        elif old_name.lower() in new_filename.lower():
            # Case-insensitive replacement
            pattern = re.compile(re.escape(old_name), re.IGNORECASE)
            new_filename = pattern.sub(new_name, new_filename)
    
    return new_filename

# ============================================================================
# PROCESS ROBoFLOW DATASET: Map classes and copy files
# ============================================================================
print("\n" + "="*80)
print("📦 PROCESSING ROBoFLOW DATASET")
print("="*80)

# Load Roboflow data.yaml to understand structure
roboflow_yaml_path = roboflow_dataset_dir / 'data.yaml'
with open(roboflow_yaml_path, 'r') as f:
    roboflow_config = yaml.safe_load(f)

roboflow_class_names = roboflow_config.get('names', [])
print(f"\n📋 Roboflow Original Classes from YAML: {roboflow_class_names}")

# Create mapping: old_class_id (from label files) → new_class_id (final)
# Label files use numeric IDs (0, 1, 2, 3), we need to map these to final IDs
roboflow_class_id_mapping = {}

# Handle different YAML formats
if isinstance(roboflow_class_names, dict):
    # Format: {0: 'Functional', 1: 'Mild', 2: 'Moderate', 3: 'Severe'}
    for old_id, class_name in roboflow_class_names.items():
        # Map class name to final class name
        if class_name in ROBOFLOW_CLASS_NAME_MAPPING:
            final_class_name = ROBOFLOW_CLASS_NAME_MAPPING[class_name]
            final_id = FINAL_CLASS_TO_ID[final_class_name]
            roboflow_class_id_mapping[str(old_id)] = str(final_id)
        else:
            # Fallback: try to use the ID mapping
            if str(old_id) in ROBOFLOW_CLASS_ID_MAPPING:
                mapped_name = ROBOFLOW_CLASS_ID_MAPPING[str(old_id)]
                final_class_name = ROBOFLOW_CLASS_NAME_MAPPING[mapped_name]
                final_id = FINAL_CLASS_TO_ID[final_class_name]
                roboflow_class_id_mapping[str(old_id)] = str(final_id)
elif isinstance(roboflow_class_names, list):
    # Format: ['Functional', 'Mild', 'Moderate', 'Severe'] or ['0', '1', '2', '3']
    for idx, class_name in enumerate(roboflow_class_names):
        # Check if it's a class name or just an ID string
        if isinstance(class_name, str) and class_name in ROBOFLOW_CLASS_NAME_MAPPING:
            # It's a class name like 'Functional'
            final_class_name = ROBOFLOW_CLASS_NAME_MAPPING[class_name]
            final_id = FINAL_CLASS_TO_ID[final_class_name]
            roboflow_class_id_mapping[str(idx)] = str(final_id)
        elif str(idx) in ROBOFLOW_CLASS_ID_MAPPING:
            # Use ID mapping (for cases where YAML has ['0', '1', '2', '3'])
            mapped_name = ROBOFLOW_CLASS_ID_MAPPING[str(idx)]
            final_class_name = ROBOFLOW_CLASS_NAME_MAPPING[mapped_name]
            final_id = FINAL_CLASS_TO_ID[final_class_name]
            roboflow_class_id_mapping[str(idx)] = str(final_id)

# Also add direct ID mappings (in case YAML doesn't have names)
for old_id, class_name in ROBOFLOW_CLASS_ID_MAPPING.items():
    if old_id not in roboflow_class_id_mapping:
        final_class_name = ROBOFLOW_CLASS_NAME_MAPPING[class_name]
        final_id = FINAL_CLASS_TO_ID[final_class_name]
        roboflow_class_id_mapping[old_id] = str(final_id)

print(f"\n🔄 Label File Class ID Mapping (for .txt files):")
for old_id, new_id in sorted(roboflow_class_id_mapping.items(), key=lambda x: int(x[0])):
    old_name = ROBOFLOW_CLASS_ID_MAPPING.get(old_id, 'Unknown')
    final_name = ROBOFLOW_CLASS_NAME_MAPPING.get(old_name, 'Unknown')
    print(f"   Label ID {old_id} ({old_name}) → Final ID {new_id} ({final_name})")

# Process each split
roboflow_stats = {'train': 0, 'valid': 0, 'test': 0}

for split in splits:
    print(f"\n📂 Processing Roboflow {split.upper()} split...")
    
    # Source paths
    src_images_dir = roboflow_dataset_dir / split / 'images'
    src_labels_dir = roboflow_dataset_dir / split / 'labels'
    
    # Destination paths
    dst_images_dir = combined_dataset_dir / split / 'images'
    dst_labels_dir = combined_dataset_dir / split / 'labels'
    
    if not src_images_dir.exists() or not src_labels_dir.exists():
        print(f"   ⚠️  Split {split} not found, skipping...")
        continue
    
    # Get all image files
    image_files = list(src_images_dir.glob('*.jpg')) + list(src_images_dir.glob('*.png')) + \
                  list(src_images_dir.glob('*.JPG')) + list(src_images_dir.glob('*.PNG'))
    
    copied_count = 0
    updated_count = 0
    
    for img_path in tqdm(image_files, desc=f"  Copying {split} images", leave=False):
        # Rename image file: replace class names in filename
        # Example: "Functional-1-_png.rf.xxx.jpg" → "Healthy-1-_png.rf.xxx.jpg"
        renamed_img_name = rename_image_with_class(img_path.name, ROBOFLOW_CLASS_NAME_MAPPING)
        
        # Use renamed filename directly (no prefix)
        dst_img_name = renamed_img_name
        dst_img_path = dst_images_dir / dst_img_name
        
        # Copy image with new name
        shutil.copy2(img_path, dst_img_path)
        
        # Find and update corresponding label file
        # Label file name should match the renamed image name
        # Get the stem (filename without extension) from the renamed image
        renamed_img_stem = Path(renamed_img_name).stem
        label_name = img_path.stem + '.txt'  # Original label name
        src_label_path = src_labels_dir / label_name
        
        # Label file should match the renamed image name
        dst_label_name = renamed_img_stem + '.txt'  # Match the new image name exactly
        dst_label_path = dst_labels_dir / dst_label_name
        
        if src_label_path.exists():
            # Copy label file with new name
            shutil.copy2(src_label_path, dst_label_path)
            
            # Update class IDs in label file
            if update_label_file(dst_label_path, roboflow_class_id_mapping):
                updated_count += 1
            
            copied_count += 1
        else:
            # If no label file, create one with default (whole image classification)
            with open(dst_label_path, 'w') as f:
                # YOLO format: class_id center_x center_y width height
                # For whole image: 0.5 0.5 1.0 1.0
                # Use first class as default (shouldn't happen, but safety)
                default_class = list(roboflow_class_id_mapping.values())[0] if roboflow_class_id_mapping else '0'
                f.write(f"{default_class} 0.5 0.5 1.0 1.0\n")
            copied_count += 1
    
    roboflow_stats[split] = copied_count
    print(f"   ✅ Copied {copied_count} images, updated {updated_count} label files")

print(f"\n✅ Roboflow dataset processing complete!")
print(f"   Train: {roboflow_stats['train']} images")
print(f"   Valid: {roboflow_stats['valid']} images")
print(f"   Test: {roboflow_stats['test']} images")


In [ ]:
# ============================================================================
# PROCESS BSED DATASET: Map classes and copy files
# ============================================================================
print("\n" + "="*80)
print("📦 PROCESSING BSED DATASET")
print("="*80)

# Load BSED data.yaml
bsed_yaml_path = bsed_dataset_dir / 'data.yaml'
with open(bsed_yaml_path, 'r') as f:
    bsed_config = yaml.safe_load(f)

print(f"\n📋 BSED Original Classes: {bsed_config.get('names', 'N/A')}")

# BSED classes: Stage1, Stage2, Stage3
# These map directly to final classes (no remapping needed, just need to get correct IDs)
bsed_class_id_mapping = {}
bsed_class_names = bsed_config.get('names', {})

if isinstance(bsed_class_names, dict):
    # Format: {0: 'Stage1', 1: 'Stage2', 2: 'Stage3'}
    for old_id, class_name in bsed_class_names.items():
        new_id = FINAL_CLASS_TO_ID[class_name]
        bsed_class_id_mapping[str(old_id)] = str(new_id)
elif isinstance(bsed_class_names, list):
    # Format: ['Stage1', 'Stage2', 'Stage3']
    for old_id, class_name in enumerate(bsed_class_names):
        new_id = FINAL_CLASS_TO_ID[class_name]
        bsed_class_id_mapping[str(old_id)] = str(new_id)

print(f"\n🔄 Class ID Mapping:")
for old_id, new_id in bsed_class_id_mapping.items():
    class_name = bsed_class_names[int(old_id)] if isinstance(bsed_class_names, dict) else bsed_class_names[int(old_id)]
    print(f"   BSED Class {old_id} ({class_name}) → Final Class ID {new_id}")

# Process each split
bsed_stats = {'train': 0, 'valid': 0, 'test': 0}

for split in splits:
    print(f"\n📂 Processing BSED {split.upper()} split...")
    
    # Source paths
    src_images_dir = bsed_dataset_dir / split / 'images'
    src_labels_dir = bsed_dataset_dir / split / 'labels'
    
    # Destination paths
    dst_images_dir = combined_dataset_dir / split / 'images'
    dst_labels_dir = combined_dataset_dir / split / 'labels'
    
    if not src_images_dir.exists() or not src_labels_dir.exists():
        print(f"   ⚠️  Split {split} not found, skipping...")
        continue
    
    # Get all image files
    image_files = list(src_images_dir.glob('*.jpg')) + list(src_images_dir.glob('*.png')) + \
                  list(src_images_dir.glob('*.JPG')) + list(src_images_dir.glob('*.PNG'))
    
    copied_count = 0
    updated_count = 0
    
    for img_path in tqdm(image_files, desc=f"  Copying {split} images", leave=False):
        # Copy image (use original name, no prefix)
        dst_img_name = img_path.name
        dst_img_path = dst_images_dir / dst_img_name
        shutil.copy2(img_path, dst_img_path)
        
        # Find and update corresponding label file
        label_name = img_path.stem + '.txt'
        src_label_path = src_labels_dir / label_name
        dst_label_name = label_name  # Use original name, no prefix
        dst_label_path = dst_labels_dir / dst_label_name
        
        if src_label_path.exists():
            # Copy label file
            shutil.copy2(src_label_path, dst_label_path)
            
            # Update class IDs in label file
            if update_label_file(dst_label_path, bsed_class_id_mapping):
                updated_count += 1
            
            copied_count += 1
        else:
            # If no label file, create one
            with open(dst_label_path, 'w') as f:
                default_class = list(bsed_class_id_mapping.values())[0] if bsed_class_id_mapping else '0'
                f.write(f"{default_class} 0.5 0.5 1.0 1.0\n")
    
    bsed_stats[split] = copied_count
    print(f"   ✅ Copied {copied_count} images, updated {updated_count} label files")

print(f"\n✅ BSED dataset processing complete!")
print(f"   Train: {bsed_stats['train']} images")
print(f"   Valid: {bsed_stats['valid']} images")
print(f"   Test: {bsed_stats['test']} images")


In [ ]:
# ============================================================================
# CREATE COMBINED DATA.YAML
# ============================================================================
print("\n" + "="*80)
print("📝 CREATING COMBINED DATA.YAML")
print("="*80)

# Create final class mapping for YAML
final_class_names = {idx: name for idx, name in enumerate(FINAL_CLASS_ORDER)}

# Create combined data.yaml
combined_yaml_config = {
    'path': str(combined_dataset_dir.absolute()),
    'train': 'train/images',
    'val': 'valid/images',
    'test': 'test/images',
    'names': final_class_names,
    'nc': len(FINAL_CLASS_ORDER)
}

# Save combined data.yaml
combined_yaml_path = combined_dataset_dir / 'data.yaml'
with open(combined_yaml_path, 'w') as f:
    yaml.dump(combined_yaml_config, f, default_flow_style=False, sort_keys=False)

print(f"\n✅ Combined data.yaml created: {combined_yaml_path}")
print(f"\n📋 Configuration:")
print(f"   Path: {combined_yaml_config['path']}")
print(f"   Train: {combined_yaml_config['train']}")
print(f"   Validation: {combined_yaml_config['val']}")
print(f"   Test: {combined_yaml_config['test']}")
print(f"   Number of classes: {combined_yaml_config['nc']}")
print(f"\n   Class mapping:")
for idx, name in final_class_names.items():
    print(f"      {idx}: {name}")

# ============================================================================
# FINAL STATISTICS
# ============================================================================
print("\n" + "="*80)
print("📊 FINAL COMBINED DATASET STATISTICS")
print("="*80)

total_stats = {
    'train': bsed_stats['train'] + roboflow_stats['train'],
    'valid': bsed_stats['valid'] + roboflow_stats['valid'],
    'test': bsed_stats['test'] + roboflow_stats['test']
}

print(f"\n📈 Total Images:")
print(f"   Training: {total_stats['train']} images")
print(f"   Validation: {total_stats['valid']} images")
print(f"   Test: {total_stats['test']} images")
print(f"   TOTAL: {sum(total_stats.values())} images")

print(f"\n📦 Source Breakdown:")
print(f"   BSED Dataset:")
print(f"      Train: {bsed_stats['train']}, Valid: {bsed_stats['valid']}, Test: {bsed_stats['test']}")
print(f"   Roboflow Dataset:")
print(f"      Train: {roboflow_stats['train']}, Valid: {roboflow_stats['valid']}, Test: {roboflow_stats['test']}")

# Verify label files match
print(f"\n🔍 Verifying label files...")
for split in splits:
    images_dir = combined_dataset_dir / split / 'images'
    labels_dir = combined_dataset_dir / split / 'labels'
    
    if images_dir.exists() and labels_dir.exists():
        images = list(images_dir.glob('*.jpg')) + list(images_dir.glob('*.png')) + \
                 list(images_dir.glob('*.JPG')) + list(images_dir.glob('*.PNG'))
        labels = list(labels_dir.glob('*.txt'))
        
        print(f"   {split.upper()}: {len(images)} images, {len(labels)} labels", end="")
        if len(images) == len(labels):
            print(" ✅")
        else:
            print(f" ⚠️  (mismatch!)")

print("\n" + "="*80)
print("✅ DATASET COMBINATION COMPLETE!")
print("="*80)
print(f"\n📁 Combined dataset location: {combined_dataset_dir}")
print(f"📄 Configuration file: {combined_yaml_path}")

# ============================================================================
# CLEANUP: Delete copied datasets to save space
# ============================================================================
print("\n" + "="*80)
print("🧹 CLEANING UP COPIED DATASETS")
print("="*80)

# Option to skip cleanup (set to False if you want to keep the copies)
DELETE_COPIED_DATASETS = True

if DELETE_COPIED_DATASETS:
    print("\n🗑️  Deleting copied datasets (already merged)...")
    
    # Delete BSED copied dataset
    if bsed_dataset_dir.exists():
        try:
            shutil.rmtree(bsed_dataset_dir)
            print(f"   ✅ Deleted: {bsed_dataset_dir}")
        except Exception as e:
            print(f"   ⚠️  Error deleting {bsed_dataset_dir}: {e}")
    else:
        print(f"   ℹ️  BSED dataset not found (already deleted or not copied)")
    
    # Delete Roboflow copied dataset
    if roboflow_dataset_dir.exists():
        try:
            shutil.rmtree(roboflow_dataset_dir)
            print(f"   ✅ Deleted: {roboflow_dataset_dir}")
        except Exception as e:
            print(f"   ⚠️  Error deleting {roboflow_dataset_dir}: {e}")
    else:
        print(f"   ℹ️  Roboflow dataset not found (already deleted or not copied)")
    
    print(f"\n✅ Cleanup complete! Only combined dataset remains.")
    print(f"   Combined dataset: {combined_dataset_dir}")
else:
    print("\n💡 Cleanup skipped (DELETE_COPIED_DATASETS = False)")
    print(f"   BSED dataset: {bsed_dataset_dir}")
    print(f"   Roboflow dataset: {roboflow_dataset_dir}")
    print(f"   Combined dataset: {combined_dataset_dir}")

print("\n" + "="*80)
print("💡 Next step: Use this dataset for training:")
print(f"   from ultralytics import YOLO")
print(f"   model = YOLO('yolov8n.pt')")
print(f"   model.train(data='{combined_yaml_path}', epochs=100)")
print("="*80)


In [ ]:
!zip -r /kaggle/working/BananaEarlyDataSets.zip /kaggle/working